<a href="https://colab.research.google.com/github/YejinShin1/YejinShin1/blob/master/%EC%8B%A0%ED%95%9C%EC%B9%B4%EB%93%9C_VSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
import vsm

In [ ]:
df = pd.read_excel('KCTI_NATIVE_2019.xlsx')
df.head()

,v1,v2,v3,gb3,gb2,sex_ccd,cln_age_r,ta_ym,daw_ccd_r,apv_ts_dl_tm_r,vlm,usec
0,세종,세종,.,취미오락,종합쇼핑,M,40,201901,WHITE,활동,1878919479,63579
1,대전,세종,.,취미오락,외식,M,20,201901,WHITE,휴식,69452328,2952
2,서울,세종,.,취미오락,외식,F,40,201901,RED,휴식,9267020,286
3,경남,세종,.,취미오락,종합쇼핑,M,50,201901,RED,휴식,603499,20
4,충북,세종,.,취미오락,종합쇼핑,F,40,201901,WHITE,활동,107638881,2379


In [ ]:
v1 = df['v1'].tolist()
v2 = df['v2'].tolist()
#v3 = df['v3'].tolist()
gb3 = df['gb3'].tolist()
gb2 = df['gb2'].tolist()
sex_ccd = df['sex_ccd'].tolist()
cln_age_r = df['cln_age_r'].tolist()
ta_ym = df['ta_ym'].tolist()
daw_ccd_r = df['daw_ccd_r'].tolist()
apv_ts_dl_tm_r = df['apv_ts_dl_tm_r'].tolist()

cln_age_r = [str(age) for age in cln_age_r]
ta_ym = [str(ym) for ym in ta_ym]

In [ ]:
v1 = [city + str("_거주지") for city in v1]
v2 = [city + str("_사용장소") for city in v2]

In [ ]:
df_list = list(zip(v1, v2, gb3, gb2, sex_ccd, cln_age_r, ta_ym, daw_ccd_r, apv_ts_dl_tm_r))
df_list = [list(user) for user in df_list]
df_list[:10]

[['세종_거주지', '세종_사용장소', '취미오락', '종합쇼핑', 'M', '40', '201901', 'WHITE', '활동'],
 ['대전_거주지', '세종_사용장소', '취미오락', '외식', 'M', '20', '201901', 'WHITE', '휴식'],
 ['서울_거주지', '세종_사용장소', '취미오락', '외식', 'F', '40', '201901', 'RED', '휴식'],
 ['경남_거주지', '세종_사용장소', '취미오락', '종합쇼핑', 'M', '50', '201901', 'RED', '휴식'],
 ['충북_거주지', '세종_사용장소', '취미오락', '종합쇼핑', 'F', '40', '201901', 'WHITE', '활동'],
 ['충북_거주지', '세종_사용장소', '취미오락', '목욕', 'M', '30', '201901', 'RED', '활동'],
 ['부산_거주지', '세종_사용장소', '취미오락', '외식', 'M', '50', '201901', 'WHITE', '휴식'],
 ['대전_거주지', '세종_사용장소', '스포츠활동', '레저스포츠', 'M', '30', '201901', 'WHITE', '활동'],
 ['세종_거주지', '세종_사용장소', '취미오락', '종합쇼핑', 'M', '20', '201901', 'WHITE', '휴식'],
 ['전북_거주지', '세종_사용장소', '취미오락', '외식', 'M', '60', '201901', 'RED', '활동']]

In [ ]:
def distinct_words(corpus):
    """ Determine a list of distinct words for the corpus.
        Params:
            corpus (list of list of strings): corpus of documents
        Return:
            corpus_words (list of strings): list of distinct words across the corpus, sorted (using python 'sorted' function)
            num_corpus_words (integer): number of distinct words across the corpus
    """
    
   
 
    corpus_words = sorted(list(set([word for words_list in corpus for word in words_list])))
    num_corpus_words = len(corpus_words)
        
    return corpus_words, num_corpus_words

In [ ]:
def compute_co_occurrence_matrix(corpus, window_size=4):
    """ Compute co-occurrence matrix for the given corpus and window_size (default of 4).
    
        Params:
            corpus (list of list of strings): corpus of documents
            window_size (int): size of context window
        Return:
            M (numpy matrix of shape (number of corpus words, number of corpus words)): 
                Co-occurence matrix of word counts. 
                The ordering of the words in the rows/columns should be the same as the ordering of the words given by the distinct_words function.
            word2Ind (dict): dictionary that maps word to index (i.e. row/column number) for matrix M.
    """
    words, num_words = distinct_words(corpus)
    M = None
    word2Ind = {}
    

    M = np.zeros((num_words, num_words))
    word2Ind = dict(zip(words, range(num_words)))
    
    for doc in corpus:
        current_idx = 0
        doc_len = len(doc)
        while current_idx < doc_len:
            left_boudary = max(current_idx-window_size, 0)
            right_boundary = min(current_idx+window_size+1, doc_len)
            outside_words = doc[left_boudary:current_idx] + doc[current_idx+1:right_boundary]
            center_word = doc[current_idx]
            center_idx = word2Ind[center_word]
            
            for outside_word in outside_words:
                outside_idx = word2Ind[outside_word]
                M[outside_idx, center_idx] += 1
            
            current_idx += 1
    
    return M, word2Ind

In [ ]:
M_co_occurrence, word2Ind_co_occurrence = compute_co_occurrence_matrix(df_list, window_size=10)

In [ ]:
M_df = pd.DataFrame(M_co_occurrence, index=word2Ind_co_occurrence.keys())
vsm.pmi(M_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79
20,0.000000,0.118219,0.110372,0.000000,0.000000,0.000000,0.000000,0.099029,0.132299,0.142258,0.097558,0.203112,0.177325,0.003070,0.150605,0.114280,0.000000,0.134815,0.022771,0.000000,0.357845,0.000000,0.197407,0.187372,0.000000,0.031219,0.165937,0.162506,0.156066,0.182242,0.274142,0.294704,0.039737,0.293024,0.048855,0.199837,0.105609,0.139557,0.348257,0.032697,0.181685,0.032333,0.039619,0.030806,0.156584,0.000000,0.032403,0.000000,0.240641,0.000000,0.000000,0.018022,0.017872,0.159281,0.099322,0.246970,0.142785,0.095898,0.168938,0.565138,0.000000,0.167088,0.000000,0.136728,0.005159,0.170681,0.063548,0.000000,0.000000,0.000000,0.122357,0.053201,0.193743,0.085240,0.137729,0.277713,0.128132,0.082622,0.040241,0.167676
201901,0.118219,0.000000,0.000000,0.116969,0.117015,0.118370,0.118679,0.118172,0.117477,0.116947,0.118458,0.115083,0.118546,0.117170,0.118318,0.118636,0.114895,0.118276,0.115029,0.111569,0.112604,0.115736,0.119159,0.115220,0.121173,0.117470,0.120352,0.118520,0.118124,0.119350,0.121196,0.115619,0.116585,0.115467,0.120812,0.117704,0.118000,0.118807,0.118036,0.117113,0.120259,0.114857,0.118993,0.119118,0.126502,0.116136,0.115703,0.126656,0.119255,0.117769,0.120698,0.119566,0.175213,0.116339,0.117423,0.117904,0.112481,0.118219,0.118824,0.117106,0.102531,0.119668,0.114514,0.118836,0.113173,0.118270,0.122532,0.119957,0.118127,0.113569,0.118288,0.115880,0.115981,0.117035,0.118447,0.118588,0.117330,0.120185,0.118028,0.117169
201902,0.110372,0.000000,0.000000,0.131447,0.130695,0.107808,0.102506,0.111178,0.122951,0.131814,0.106299,0.162390,0.104789,0.128096,0.108685,0.103241,0.165415,0.109407,0.163259,0.217443,0.201570,0.151797,0.094207,0.160183,0.058617,0.123056,0.073291,0.105239,0.111999,0.090903,0.058211,0.153701,0.137834,0.156167,0.065108,0.119117,0.114111,0.100298,0.113498,0.129050,0.074940,0.166027,0.097085,0.094926,0.000000,0.145237,0.152325,0.000000,0.092554,0.118024,0.067131,0.087125,0.000000,0.141900,0.123852,0.115742,0.203469,0.110386,0.100008,0.129168,0.345770,0.085351,0.171518,0.099794,0.192712,0.109503,0.033818,0.080278,0.111940,0.186510,0.109199,0.149443,0.147790,0.130348,0.106486,0.104069,0.125413,0.076246,0.113626,0.128110
30,0.000000,0.116969,0.131447,0.000000,0.000000,0.000000,0.000000,0.083711,0.143804,0.144558,0.095611,0.117347,0.098035,0.099274,0.126509,0.101640,0.125356,0.128160,0.117874,0.084224,0.180160,0.126837,0.130043,0.129476,0.045006,0.108097,0.123223,0.151883,0.138600,0.108412,0.128120,0.244812,0.058404,0.164446,0.135689,0.169124,0.112285,0.135176,0.194707,0.117693,0.090398,0.204802,0.181881,0.129657,0.252938,0.110192,0.158149,0.000000,0.210529,0.149839,0.117173,0.074541,0.741667,0.123742,0.180742,0.072690,0.129682,0.129307,0.126457,0.093644,0.036413,0.075432,0.101803,0.111493,0.123562,0.094230,0.121745,0.000000,0.095964,0.268950,0.132456,0.136090,0.113250,0.098446,0.097663,0.125448,0.120478,0.143398,0.111388,0.124439
40,0.000000,0.117015,0.130695,0.000000,0.000000,0.000000,0.000000,0.122930,0.113714,0.116201,0.119060,0.076707,0.150634,0.138736,0.103062,0.130830,0.188807,0.105284,0.160354,0.406538,0.072504,0.235836,0.096327,0.096212,0.347373,0.121396,0.085390,0.106229,0.130906,0.086779,0.116641,0.136539,0.103748,0.095258,0.183690,0.094085,0.083882,0.117802,0.043660,0.100511,0.036260,0.295561,0.147558,0.199357,0.272175,0.171108,0.203385,0.282799,0.003869,0.207073,0.198487,0.088851,0.562502,0.104831,0.123287,0.084459,0.131067,0.114168,0.078636,0.000000,0.258873,0.116884,0.208271,0.115603,0.190872,0.167600,0.143973,0.178312,0.105665,0.277691,0.133616,0.157478,0.104744,0.140329,0.082211,0.062339,0.074790,0.139522,0.136348,0.108252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [ ]:
pd.set_option('display.max_row', 80)
df_1 = vsm.neighbors('서울_거주지', vsm.pmi(M_df))
df_1.to_excel('통합_서울_거주지_pmi.xlsx')

In [ ]:
df_2 = vsm.neighbors('서울_사용장소', vsm.pmi(M_df))
df_2.to_excel('통합_서울_사용장소_pmi.xlsx')

In [ ]:
df_3 = vsm.neighbors('인천_거주지', vsm.pmi(M_df))
df_3.to_excel('통합_인천_거주지_pmi.xlsx')

In [ ]:
df_4 = vsm.neighbors('인천_사용장소', vsm.pmi(M_df))
df_4.to_excel('통합_인천_사용장소_pmi.xlsx')

In [ ]:
df_5 = vsm.neighbors('경기_거주지', vsm.pmi(M_df))
df_5.to_excel('통합_경기_거주지_pmi.xlsx')

In [ ]:
df_6 = vsm.neighbors('경기_사용장소', vsm.pmi(M_df))
df_6.to_excel('통합_경기_사용장소_pmi.xlsx')

In [ ]:
df_7 = vsm.neighbors('전북_거주지', vsm.pmi(M_df))
df_7.to_excel('통합_전북_거주지_pmi.xlsx')

In [ ]:
df_8 = vsm.neighbors('전북_사용장소', vsm.pmi(M_df))
df_8.to_excel('통합_전북_사용장소_pmi.xlsx')

In [ ]:
df_9 = vsm.neighbors('전남_거주지', vsm.pmi(M_df))
df_9.to_excel('통합_전남_거주지_pmi.xlsx')

In [ ]:
df_10 = vsm.neighbors('전남_사용장소', vsm.pmi(M_df))
df_10.to_excel('통합_전남_사용장소_pmi.xlsx')

In [ ]:
df_11 = vsm.neighbors('대전_거주지', vsm.pmi(M_df))
df_11.to_excel('통합_대전_거주지_pmi.xlsx')

In [ ]:
df_12 = vsm.neighbors('대전_사용장소', vsm.pmi(M_df))
df_12.to_excel('통합_대전_사용장소_pmi.xlsx')

In [ ]:
df_13 = vsm.neighbors('세종_거주지', vsm.pmi(M_df))
df_13.to_excel('통합_세종_거주지_pmi.xlsx')

In [ ]:
df_14 = vsm.neighbors('세종_사용장소', vsm.pmi(M_df))
df_14.to_excel('통합_세종_사용장소_pmi.xlsx')

In [ ]:
df_15 = vsm.neighbors('대구_거주지', vsm.pmi(M_df))
df_15.to_excel('통합_대구_거주지_pmi.xlsx')

In [ ]:
df_16 = vsm.neighbors('대구_사용장소', vsm.pmi(M_df))
df_16.to_excel('통합_대구_사용장소_pmi.xlsx')

In [ ]:
df_17 = vsm.neighbors('부산_사용장소', vsm.pmi(M_df))
df_17.to_excel('통합_부산_사용장소_pmi.xlsx')

In [ ]:
df_18 = vsm.neighbors('부산_거주지', vsm.pmi(M_df))
df_18.to_excel('통합_부산_거주지_pmi.xlsx')

In [ ]:
df_19 = vsm.neighbors('충북_거주지', vsm.pmi(M_df))
df_19.to_excel('통합_충북_거주지_pmi.xlsx')

In [ ]:
df_20 = vsm.neighbors('충북_사용장소', vsm.pmi(M_df))
df_20.to_excel('통합_충북_사용장소_pmi.xlsx')

In [ ]:
df_21 = vsm.neighbors('충남_거주지', vsm.pmi(M_df))
df_21.to_excel('통합_충남_거주지_pmi.xlsx')

In [ ]:
df_22 = vsm.neighbors('충남_사용장소', vsm.pmi(M_df))
df_22.to_excel('통합_충남_사용장소_pmi.xlsx')

In [ ]:
df_23 = vsm.neighbors('제주_사용장소', vsm.pmi(M_df))
df_23.to_excel('통합_제주_사용장소_pmi.xlsx')

In [ ]:
df_24 = vsm.neighbors('제주_거주지', vsm.pmi(M_df))
df_24.to_excel('통합_제주_거주지_pmi.xlsx')

In [ ]:
df_25 = vsm.neighbors('광주_사용장소', vsm.pmi(M_df))
df_25.to_excel('통합_광주_사용장소_pmi.xlsx')

In [ ]:
df_26 = vsm.neighbors('광주_거주지', vsm.pmi(M_df))
df_26.to_excel('통합_광주_거주지_pmi.xlsx')

In [ ]:
df_27 = vsm.neighbors('강원_거주지', vsm.pmi(M_df))
df_27.to_excel('통합_강원_거주지_pmi.xlsx')

In [ ]:
df_28 = vsm.neighbors('강원_사용장소', vsm.pmi(M_df))
df_28.to_excel('통합_강원_사용장소_pmi.xlsx')

In [ ]:
df_29 = vsm.neighbors('경남_사용장소', vsm.pmi(M_df))
df_29.to_excel('통합_경남_사용장소_pmi.xlsx')

In [ ]:
df_30 = vsm.neighbors('경남_거주지', vsm.pmi(M_df))
df_30.to_excel('통합_경남_거주지_pmi.xlsx')

In [ ]:
df_31 = vsm.neighbors('경북_거주지', vsm.pmi(M_df))
df_31.to_excel('통합_경북_거주지_pmi.xlsx')

In [ ]:
df_32 = vsm.neighbors('경북_사용장소', vsm.pmi(M_df))
df_32.to_excel('통합_경북_사용장소_pmi.xlsx')

In [ ]:
df_33 = vsm.neighbors('울산_사용장소', vsm.pmi(M_df))
df_33.to_excel('통합_울산_사용장소_pmi.xlsx')

In [ ]:
df_34 = vsm.neighbors('울산_거주지', vsm.pmi(M_df))
df_34.to_excel('통합_울산_거주지_pmi.xlsx')

In [ ]:
df_svd = vsm.lsa(M_df, k=5)

In [ ]:
df_35 = vsm.neighbors('인천_거주지', df_svd)
df_35.to_excel('통합_인천_거주지_svd.xlsx')
df_36 = vsm.neighbors('인천_사용장소', df_svd)
df_36.to_excel('통합_인천_사용장소_svd.xlsx')
df_37 = vsm.neighbors('서울_거주지', df_svd)
df_37.to_excel('통합_서울_거주지_svd.xlsx')
df_38 = vsm.neighbors('서울_사용장소', df_svd)
df_38.to_excel('통합_서울_사용장소_svd.xlsx')
df_39 = vsm.neighbors('경기_거주지', df_svd)
df_39.to_excel('통합_경기_사용장소_svd.xlsx')
df_40 = vsm.neighbors('경기_사용장소', df_svd)
df_40.to_excel('통합_경기_사용장소_svd.xlsx')
df_41 = vsm.neighbors('전북_거주지', df_svd)
df_41.to_excel('통합_전북_거주지_svd.xlsx')
df_42 = vsm.neighbors('전북_사용장소', df_svd)
df_42.to_excel('통합_전북_사용장소_svd.xlsx')

In [ ]:
df_43 = vsm.neighbors('대전_거주지', df_svd)
df_43.to_excel('통합_대전_거주지_svd.xlsx')
df_44 = vsm.neighbors('대전_사용장소', df_svd)
df_44.to_excel('통합_대전_사용장소_svd.xlsx')
df_45 = vsm.neighbors('대구_거주지', df_svd)
df_45.to_excel('통합_대구_거주지_svd.xlsx')
df_46 = vsm.neighbors('대구_사용장소', df_svd)
df_46.to_excel('통합_대구_사용장소_svd.xlsx')
df_47 = vsm.neighbors('부산_거주지', df_svd)
df_47.to_excel('통합_부산_거주지_svd.xlsx')
df_48 = vsm.neighbors('부산_거주지', df_svd)
df_48.to_excel('통합_부산_거주지_svd.xlsx')
df_49 = vsm.neighbors('충북_거주지', df_svd)
df_49.to_excel('통합_충북_거주지_svd.xlsx')
df_50 = vsm.neighbors('충북_사용장소', df_svd)
df_50.to_excel('통합_충북_사용장소_svd.xlsx')
df_51 = vsm.neighbors('충남_거주지', df_svd)
df_51.to_excel('통합_충남_거주지_svd.xlsx')
df_52 = vsm.neighbors('충남_사용장소', df_svd)
df_52.to_excel('통합_충남_사용장소_svd.xlsx')
df_53 = vsm.neighbors('제주_거주지', df_svd)
df_53.to_excel('통합_제주_거주지_svd.xlsx')
df_54 = vsm.neighbors('제주_사용장소', df_svd)
df_54.to_excel('통합_제주_사용장소_svd.xlsx')

In [ ]:
df_55 = vsm.neighbors('광주_거주지', df_svd)
df_55.to_excel('통합_광주_거주지_svd.xlsx')
df_56 = vsm.neighbors('광주_사용장소', df_svd)
df_56.to_excel('통합_광주_사용장소_svd.xlsx')
df_57 = vsm.neighbors('세종_거주지', df_svd)
df_57.to_excel('통합_세종_거주지_svd.xlsx')
df_58 = vsm.neighbors('세종_사용장소', df_svd)
df_58.to_excel('통합_세종_사용장소_svd.xlsx')
df_59 = vsm.neighbors('강원_거주지', df_svd)
df_59.to_excel('통합_강원_거주지_svd.xlsx')
df_60 = vsm.neighbors('강원_사용장소', df_svd)
df_60.to_excel('통합_강원_사용장소_svd.xlsx')
df_61 = vsm.neighbors('경남_거주지', df_svd)
df_61.to_excel('통합_경남_거주지_svd.xlsx')
df_62 = vsm.neighbors('경남_사용장소', df_svd)
df_62.to_excel('통합_경남_사용장소_svd.xlsx')
df_63 = vsm.neighbors('전남_거주지', df_svd)
df_63.to_excel('통합_전남_거주지_svd.xlsx')
df_64 = vsm.neighbors('전남_사용장소', df_svd)
df_64.to_excel('통합_전남_사용장소_svd.xlsx')
df_65 = vsm.neighbors('경북_거주지', df_svd)
df_65.to_excel('통합_경북_거주지_svd.xlsx')
df_66 = vsm.neighbors('경북_사용장소', df_svd)
df_66.to_excel('통합_경북_사용장소_svd.xlsx')
df_67 = vsm.neighbors('울산_거주지', df_svd)
df_67.to_excel('통합_울산_거주지_svd.xlsx')
df_68 = vsm.neighbors('울산_사용장소', df_svd)
df_68.to_excel('통합_울산_사용장소_svd.xlsx')

In [ ]:
df_69 = vsm.neighbors('부산_사용장소', df_svd)
df_69.to_excel('통합_부산_사용장소_svd.xlsx')

In [ ]:
from google.colab import drive
drive.mount('./MyDrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at ./MyDrive


In [ ]:
before_covid = df[df['ta_ym'] != 202002]
after_covid = df[df['ta_ym'] == 202002]

In [ ]:
b_v1 = before_covid['v1'].tolist()
b_v2 = before_covid['v2'].tolist()
#v3 = df['v3'].tolist()
b_gb3 = before_covid['gb3'].tolist()
b_gb2 = before_covid['gb2'].tolist()
b_sex_ccd = before_covid['sex_ccd'].tolist()
b_cln_age_r = before_covid['cln_age_r'].tolist()
b_ta_ym = before_covid['ta_ym'].tolist()
b_daw_ccd_r = before_covid['daw_ccd_r'].tolist()
b_apv_ts_dl_tm_r = before_covid['apv_ts_dl_tm_r'].tolist()

b_cln_age_r = [str(age) for age in b_cln_age_r]
b_ta_ym = [str(ym) for ym in b_ta_ym]

In [ ]:
b_list = list(zip(b_v1, b_v2, b_gb3, b_gb2, b_sex_ccd,
                  b_cln_age_r, b_ta_ym, b_daw_ccd_r, b_apv_ts_dl_tm_r))
b_list = [list(customer) for customer in b_list]
b_list[:10]

[['경기_거주지', '세종_사용장소', '취미오락', '유흥', 'M', '40', '202001', 'RED', '휴식'],
 ['경남_거주지', '세종_사용장소', '스포츠활동', '레저스포츠', 'M', '30', '202001', 'RED', '활동'],
 ['광주_거주지', '세종_사용장소', '취미오락', '패션쇼핑', 'M', '30', '202001', 'RED', '휴식'],
 ['인천_거주지', '세종_사용장소', '취미오락', '인터넷게임', 'M', '30', '202001', 'RED', '취침'],
 ['경기_거주지', '세종_사용장소', '취미오락', '유흥', 'M', '40', '202001', 'RED', '취침'],
 ['서울_거주지', '세종_사용장소', '스포츠활동', '스포츠용품구매', 'M', '40', '202001', 'RED', '활동'],
 ['제주_거주지', '세종_사용장소', '스포츠활동', '레저스포츠', 'M', '50', '202001', 'RED', '활동'],
 ['서울_거주지', '세종_사용장소', '취미오락', '목욕', 'M', '20', '202001', 'WHITE', '취침'],
 ['강원_거주지', '세종_사용장소', '스포츠활동', '레저스포츠', 'F', '20', '202001', 'RED', '활동'],
 ['세종_거주지', '세종_사용장소', '취미오락', '독서', 'M', '60', '202001', 'RED', '휴식']]

In [ ]:
M_b_co_occurrence, b_word2Ind_co_occurrence = compute_co_occurrence_matrix(b_list, window_size=4)

In [ ]:
b_df = pd.DataFrame(M_b_co_occurrence, index=b_word2Ind_co_occurrence.keys())
vsm.pmi(b_df)

,0,1,2,3,4,5,6,7,8,9,...,69,70,71,72,73,74,75,76,77,78
20,0.000000,0.213574,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.412185,0.382768,...,0.000000,0.333966,0.000000,0.364418,0.244634,0.789539,0.083361,0.026960,0.538471,0.670351
202001,0.213574,0.000000,0.213574,0.213574,0.213574,0.213574,0.080043,0.080043,0.550046,0.550046,...,0.000000,0.000000,0.000000,0.000000,0.367725,0.773190,0.213574,0.213574,0.773190,0.773190
30,0.000000,0.213574,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.421767,0.374863,...,0.000000,0.400782,0.000000,0.391717,0.196425,0.634204,0.056337,0.067623,0.609398,0.628055
40,0.000000,0.213574,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.401606,0.391338,...,0.000000,0.430070,0.000000,0.418682,0.176801,0.553217,0.035416,0.088231,0.640706,0.607696
50,0.000000,0.213574,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.377774,0.410056,...,0.000000,0.416404,0.000000,0.411788,0.208100,0.560615,0.048828,0.129972,0.635374,0.613428
60,0.000000,0.213574,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.342778,0.436156,...,0.000000,0.404799,0.000000,0.395505,0.250709,0.451741,0.075549,0.000000,0.702100,0.539464
F,0.000000,0.080043,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.253841,0.269082,...,0.448127,0.220528,0.473856,0.225866,0.110553,0.313427,0.070619,0.000000,0.531152,0.468572
M,0.000000,0.080043,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.268976,0.257082,...,0.513782,0.293887,0.494521,0.289992,0.055530,0.602013,0.000000,0.010312,0.448322,0.498548
RED,0.412185,0.550046,0.421767,0.401606,0.377774,0.342778,0.253841,0.268976,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.951226,0.382594,0.179252,0.976153,0.925960
WHITE,0.382768,0.550046,0.374863,0.391338,0.410056,0.436156,0.269082,0.257082,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.958902,0.406334,0.539721,0.938810,0.978373


In [ ]:
pd.set_option('display.max_row', 79)
vsm.neighbors('서울_거주지', vsm.pmi(b_df))

서울_거주지     0.000000
경기_거주지     0.004422
인천_거주지     0.061960
부산_거주지     0.122167
강원_거주지     0.124138
충남_거주지     0.127497
경남_거주지     0.141496
충북_거주지     0.145517
경북_거주지     0.157736
대전_거주지     0.177496
대구_거주지     0.179192
전북_거주지     0.183708
전남_거주지     0.204094
광주_거주지     0.208047
울산_거주지     0.246093
제주_거주지     0.305267
40         0.320641
세종_거주지     0.325226
50         0.351012
60         0.385052
20         0.407107
30         0.409629
WHITE      0.450051
문화예술활동     0.489389
스포츠활동      0.501580
202001     0.518020
취미오락       0.543247
서울_사용장소    0.547946
RED        0.566052
경기_사용장소    0.588694
여행         0.588868
제주_사용장소    0.598815
인천_사용장소    0.622516
레저스포츠      0.630933
스포츠용품구매    0.644579
골프         0.645723
자전거        0.648960
대구_사용장소    0.649320
광주_사용장소    0.652552
강원_사용장소    0.652818
대전_사용장소    0.653849
세종_사용장소    0.656411
헬스         0.657936
미술공예참여     0.662546
공연관람       0.665599
유흥         0.665898
사진촬영       0.667508
목욕         0.668958
숙박         0.671197
부산_사용장소    0.677976


In [ ]:
vsm.neighbors('서울_사용장소', vsm.pmi(b_df))

서울_사용장소    0.000000
인천_사용장소    0.152934
대전_사용장소    0.161397
경기_사용장소    0.173237
제주_사용장소    0.177974
광주_사용장소    0.192777
부산_사용장소    0.201851
세종_사용장소    0.212219
대구_사용장소    0.228499
충북_사용장소    0.253289
울산_사용장소    0.265084
충남_사용장소    0.274110
강원_사용장소    0.278904
경남_사용장소    0.304054
경북_사용장소    0.307775
전남_사용장소    0.313519
취미오락       0.324372
F          0.374222
패션쇼핑       0.382395
인터넷게임      0.382739
전북_사용장소    0.387248
미용         0.402040
M          0.402781
종합쇼핑       0.403100
독서         0.409620
외식         0.414023
문화예술활동     0.421078
여행         0.430789
목욕         0.436754
유흥         0.438044
교육훈련       0.448556
공연관람       0.469095
숙박         0.479786
스포츠용품구매    0.494975
애완동물돌보기    0.501465
교통         0.502201
종교활동       0.511699
레저스포츠      0.514737
체험         0.522784
여행사        0.529637
사진촬영       0.532247
WHITE      0.540935
관광쇼핑       0.541563
경기_거주지     0.543160
스키         0.547139
서울_거주지     0.547946
골프         0.552977
헬스         0.554010
미술공예참여     0.557445
음악감상       0.596019


In [ ]:
vsm.neighbors('경기_거주지', vsm.pmi(b_df))

경기_거주지     0.000000
서울_거주지     0.004422
인천_거주지     0.055925
부산_거주지     0.131660
충남_거주지     0.134854
강원_거주지     0.135762
경남_거주지     0.150225
충북_거주지     0.153007
경북_거주지     0.161790
대전_거주지     0.178872
대구_거주지     0.181770
전북_거주지     0.187051
전남_거주지     0.215381
광주_거주지     0.218956
울산_거주지     0.264373
40         0.318281
제주_거주지     0.331392
세종_거주지     0.334310
50         0.360314
60         0.394434
30         0.401085
20         0.420392
WHITE      0.443726
문화예술활동     0.487381
스포츠활동      0.490758
202001     0.513763
서울_사용장소    0.543160
취미오락       0.552345
RED        0.560130
경기_사용장소    0.593247
여행         0.593739
제주_사용장소    0.596049
인천_사용장소    0.603608
레저스포츠      0.634919
대구_사용장소    0.636473
골프         0.644732
스포츠용품구매    0.650190
광주_사용장소    0.651913
대전_사용장소    0.654096
세종_사용장소    0.654597
자전거        0.655433
강원_사용장소    0.657640
헬스         0.666334
미술공예참여     0.673197
유흥         0.674660
공연관람       0.675168
목욕         0.675631
숙박         0.677307
사진촬영       0.678237
부산_사용장소    0.679112


In [ ]:
vsm.neighbors('경기_사용장소', vsm.pmi(b_df))

경기_사용장소    0.000000
강원_사용장소    0.143251
서울_사용장소    0.173237
인천_사용장소    0.175119
충북_사용장소    0.177831
대전_사용장소    0.186371
세종_사용장소    0.187986
충남_사용장소    0.196579
광주_사용장소    0.204970
대구_사용장소    0.229293
부산_사용장소    0.246246
경남_사용장소    0.260900
울산_사용장소    0.266261
경북_사용장소    0.277053
전남_사용장소    0.283326
취미오락       0.318855
제주_사용장소    0.321455
F          0.363691
패션쇼핑       0.365661
인터넷게임      0.369352
전북_사용장소    0.374876
M          0.380484
독서         0.381308
미용         0.386766
스포츠활동      0.394964
목욕         0.398956
유흥         0.400174
종합쇼핑       0.410299
교육훈련       0.410658
외식         0.425387
스포츠용품구매    0.434549
레저스포츠      0.446923
애완동물돌보기    0.450079
종교활동       0.468139
공연관람       0.477013
숙박         0.483399
헬스         0.483431
골프         0.484328
스키         0.494266
문화예술활동     0.515374
체험         0.519506
교통         0.521597
사진촬영       0.523363
여행사        0.532001
관광쇼핑       0.544839
미술공예참여     0.549891
자전거        0.569051
음악감상       0.580048
악기연주       0.585315
서울_거주지     0.588694


In [ ]:
vsm.neighbors('인천_거주지', vsm.pmi(b_df))

인천_거주지     0.000000
경기_거주지     0.055925
서울_거주지     0.061960
강원_거주지     0.097186
충남_거주지     0.106274
충북_거주지     0.119606
대전_거주지     0.120703
전북_거주지     0.121293
부산_거주지     0.137127
경북_거주지     0.144473
경남_거주지     0.146517
대구_거주지     0.150456
광주_거주지     0.155859
전남_거주지     0.158618
제주_거주지     0.229094
울산_거주지     0.242468
세종_거주지     0.252774
40         0.295660
30         0.325617
50         0.352655
60         0.360424
20         0.387047
스포츠활동      0.484807
WHITE      0.505294
취미오락       0.521850
202001     0.536973
RED        0.544741
여행         0.571239
문화예술활동     0.572631
인천_사용장소    0.591082
서울_사용장소    0.607482
레저스포츠      0.621650
경기_사용장소    0.624292
스포츠용품구매    0.638548
대구_사용장소    0.639820
유흥         0.640527
골프         0.641721
자전거        0.646367
제주_사용장소    0.646381
목욕         0.650342
숙박         0.651025
애완동물돌보기    0.653685
전북_사용장소    0.653737
관광쇼핑       0.654002
강원_사용장소    0.657830
미술공예참여     0.658728
헬스         0.667147
사진촬영       0.668384
미용         0.669230
공연관람       0.670813


In [ ]:
vsm.neighbors('인천_사용장소', vsm.pmi(b_df))

인천_사용장소    0.000000
서울_사용장소    0.152934
경기_사용장소    0.175119
전북_사용장소    0.189001
대전_사용장소    0.191248
대구_사용장소    0.195974
부산_사용장소    0.218544
충북_사용장소    0.222254
충남_사용장소    0.223485
제주_사용장소    0.232699
광주_사용장소    0.243606
세종_사용장소    0.252977
강원_사용장소    0.273174
경남_사용장소    0.283309
전남_사용장소    0.295646
경북_사용장소    0.303290
울산_사용장소    0.318495
취미오락       0.333819
F          0.378101
패션쇼핑       0.382991
인터넷게임      0.387567
M          0.396995
독서         0.400957
미용         0.405835
목욕         0.414215
유흥         0.415317
종합쇼핑       0.426019
교육훈련       0.431034
외식         0.440434
숙박         0.460432
애완동물돌보기    0.464871
여행         0.467626
스포츠용품구매    0.471860
스포츠활동      0.477089
레저스포츠      0.479142
종교활동       0.484192
교통         0.502200
공연관람       0.502740
체험         0.503177
여행사        0.505433
골프         0.518741
헬스         0.519251
스키         0.522156
관광쇼핑       0.527235
사진촬영       0.547870
문화예술활동     0.548892
미술공예참여     0.576288
인천_거주지     0.591082
WHITE      0.598319
음악감상       0.600726


In [ ]:
vsm.neighbors('충남_거주지', vsm.pmi(b_df))

충남_거주지     0.000000
충북_거주지     0.052325
대전_거주지     0.053475
전북_거주지     0.101031
강원_거주지     0.105651
인천_거주지     0.106274
세종_거주지     0.111422
서울_거주지     0.127497
경기_거주지     0.134854
경북_거주지     0.172038
광주_거주지     0.174026
전남_거주지     0.174718
경남_거주지     0.174837
부산_거주지     0.183802
대구_거주지     0.193252
제주_거주지     0.213159
울산_거주지     0.227067
40         0.301949
50         0.318477
60         0.326539
20         0.345132
30         0.357119
취미오락       0.472699
스포츠활동      0.563800
레저스포츠      0.573402
여행         0.583403
WHITE      0.588160
목욕         0.589567
유흥         0.593340
애완동물돌보기    0.596611
골프         0.596667
202001     0.604530
미용         0.609183
스포츠용품구매    0.614734
자전거        0.626293
RED        0.628041
숙박         0.628060
헬스         0.628387
체험         0.633053
공연관람       0.635170
종합쇼핑       0.643796
문화예술활동     0.647314
사진촬영       0.650192
외식         0.653246
종교활동       0.656141
M          0.660624
F          0.660840
경기_사용장소    0.661900
인터넷게임      0.668306
교육훈련       0.671333


In [ ]:
vsm.neighbors('충북_사용장소', vsm.pmi(b_df))

충북_사용장소    0.000000
충남_사용장소    0.085658
세종_사용장소    0.138755
대전_사용장소    0.144144
경북_사용장소    0.173862
경기_사용장소    0.177831
인천_사용장소    0.222254
경남_사용장소    0.227413
강원_사용장소    0.228583
대구_사용장소    0.249007
서울_사용장소    0.253289
부산_사용장소    0.266551
전남_사용장소    0.269517
울산_사용장소    0.285979
취미오락       0.296176
광주_사용장소    0.308331
패션쇼핑       0.317646
제주_사용장소    0.320618
전북_사용장소    0.330461
인터넷게임      0.331320
F          0.338093
M          0.342982
종합쇼핑       0.346006
미용         0.348074
외식         0.358469
독서         0.363046
목욕         0.378574
유흥         0.386198
교육훈련       0.404439
스포츠용품구매    0.419027
레저스포츠      0.438519
스키         0.439349
숙박         0.443109
애완동물돌보기    0.447199
종교활동       0.459274
체험         0.476501
교통         0.490042
공연관람       0.492458
골프         0.492520
헬스         0.509503
관광쇼핑       0.513244
여행사        0.524319
여행         0.531816
스포츠활동      0.553832
사진촬영       0.567542
미술공예참여     0.595959
운동경기관람     0.607548
자전거        0.613499
문화예술활동     0.616144
음악감상       0.655238


In [ ]:
vsm.neighbors('충남_사용장소', vsm.pmi(b_df))

충남_사용장소    0.000000
충북_사용장소    0.085658
대전_사용장소    0.136099
세종_사용장소    0.140722
경기_사용장소    0.196579
전남_사용장소    0.198184
인천_사용장소    0.223485
경북_사용장소    0.244909
경남_사용장소    0.246929
광주_사용장소    0.256888
강원_사용장소    0.267553
서울_사용장소    0.274110
전북_사용장소    0.278709
부산_사용장소    0.305314
대구_사용장소    0.306329
취미오락       0.320395
패션쇼핑       0.337686
울산_사용장소    0.344004
제주_사용장소    0.344761
인터넷게임      0.348918
M          0.355446
F          0.357420
종합쇼핑       0.361880
미용         0.363472
독서         0.373413
외식         0.373573
목욕         0.393597
유흥         0.404676
교육훈련       0.411091
스포츠용품구매    0.426557
숙박         0.437982
레저스포츠      0.441367
스키         0.456998
애완동물돌보기    0.463390
체험         0.464760
종교활동       0.481454
교통         0.486731
골프         0.491361
공연관람       0.493300
헬스         0.508785
관광쇼핑       0.516748
여행         0.518729
여행사        0.519679
스포츠활동      0.540506
사진촬영       0.570818
운동경기관람     0.591854
미술공예참여     0.603272
자전거        0.620281
악기연주       0.644646
음악감상       0.647619


In [ ]:
vsm.neighbors('충북_거주지', vsm.pmi(b_df))

충북_거주지     0.000000
대전_거주지     0.049707
충남_거주지     0.052325
강원_거주지     0.082516
세종_거주지     0.093935
인천_거주지     0.119606
전북_거주지     0.133631
경북_거주지     0.140969
서울_거주지     0.145517
경기_거주지     0.153007
대구_거주지     0.163542
경남_거주지     0.179526
부산_거주지     0.182207
제주_거주지     0.195487
광주_거주지     0.199655
울산_거주지     0.209105
전남_거주지     0.211880
40         0.295818
50         0.316643
60         0.318491
30         0.339644
20         0.341983
취미오락       0.474538
스포츠활동      0.566815
레저스포츠      0.570882
여행         0.572196
유흥         0.580901
골프         0.581609
애완동물돌보기    0.582840
목욕         0.583225
미용         0.602276
WHITE      0.614297
숙박         0.617817
202001     0.619716
RED        0.623316
스포츠용품구매    0.623524
자전거        0.626217
체험         0.629580
종합쇼핑       0.641855
외식         0.641919
종교활동       0.645944
헬스         0.646520
사진촬영       0.648218
공연관람       0.649332
F          0.652338
문화예술활동     0.653280
M          0.656115
미술공예참여     0.660352
인터넷게임      0.663301
교육훈련       0.665015


In [ ]:
vsm.neighbors('충북_사용장소', vsm.pmi(b_df))

충북_사용장소    0.000000
충남_사용장소    0.085658
세종_사용장소    0.138755
대전_사용장소    0.144144
경북_사용장소    0.173862
경기_사용장소    0.177831
인천_사용장소    0.222254
경남_사용장소    0.227413
강원_사용장소    0.228583
대구_사용장소    0.249007
서울_사용장소    0.253289
부산_사용장소    0.266551
전남_사용장소    0.269517
울산_사용장소    0.285979
취미오락       0.296176
광주_사용장소    0.308331
패션쇼핑       0.317646
제주_사용장소    0.320618
전북_사용장소    0.330461
인터넷게임      0.331320
F          0.338093
M          0.342982
종합쇼핑       0.346006
미용         0.348074
외식         0.358469
독서         0.363046
목욕         0.378574
유흥         0.386198
교육훈련       0.404439
스포츠용품구매    0.419027
레저스포츠      0.438519
스키         0.439349
숙박         0.443109
애완동물돌보기    0.447199
종교활동       0.459274
체험         0.476501
교통         0.490042
공연관람       0.492458
골프         0.492520
헬스         0.509503
관광쇼핑       0.513244
여행사        0.524319
여행         0.531816
스포츠활동      0.553832
사진촬영       0.567542
미술공예참여     0.595959
운동경기관람     0.607548
자전거        0.613499
문화예술활동     0.616144
음악감상       0.655238


In [ ]:
vsm.neighbors('세종_거주지', vsm.pmi(b_df))

세종_거주지     0.000000
대전_거주지     0.054307
충북_거주지     0.093935
충남_거주지     0.111422
전북_거주지     0.160252
제주_거주지     0.202165
강원_거주지     0.226579
광주_거주지     0.228717
인천_거주지     0.252774
대구_거주지     0.259219
경북_거주지     0.268687
전남_거주지     0.269322
경남_거주지     0.311048
울산_거주지     0.311846
부산_거주지     0.317377
서울_거주지     0.325226
40         0.325628
60         0.331029
경기_거주지     0.334310
30         0.337993
50         0.358139
20         0.383525
취미오락       0.505235
애완동물돌보기    0.537769
목욕         0.544559
유흥         0.552847
미용         0.558497
레저스포츠      0.573824
교육훈련       0.584168
스포츠활동      0.611255
독서         0.611479
자전거        0.619959
골프         0.627242
인터넷게임      0.630211
여행         0.631666
공연관람       0.633560
헬스         0.639730
종합쇼핑       0.639994
패션쇼핑       0.641539
사진촬영       0.647844
스포츠용품구매    0.654363
F          0.654656
외식         0.656339
M          0.658043
숙박         0.659654
악기연주       0.670570
교통         0.671830
체험         0.673549
종교활동       0.682109
미술공예참여     0.710886


In [ ]:
vsm.neighbors('세종_사용장소', vsm.pmi(b_df))

세종_사용장소    0.000000
대전_사용장소    0.064169
충북_사용장소    0.138755
충남_사용장소    0.140722
경기_사용장소    0.187986
서울_사용장소    0.212219
광주_사용장소    0.241289
인천_사용장소    0.252977
대구_사용장소    0.269054
부산_사용장소    0.331985
취미오락       0.332569
울산_사용장소    0.347963
경북_사용장소    0.360356
경남_사용장소    0.365755
전남_사용장소    0.373854
강원_사용장소    0.375687
종합쇼핑       0.390654
제주_사용장소    0.392557
외식         0.394723
전북_사용장소    0.395536
패션쇼핑       0.405083
인터넷게임      0.413572
F          0.414432
M          0.416369
미용         0.431204
독서         0.454804
스키         0.485356
목욕         0.487492
스포츠용품구매    0.496741
공연관람       0.505220
교육훈련       0.505743
유흥         0.508745
레저스포츠      0.519772
스포츠활동      0.552366
골프         0.562260
숙박         0.569744
종교활동       0.573197
애완동물돌보기    0.573859
문화예술활동     0.574009
교통         0.575896
체험         0.587757
헬스         0.593877
사진촬영       0.599717
미술공예참여     0.620765
운동경기관람     0.622774
관광쇼핑       0.624921
WHITE      0.639883
여행사        0.643409
경기_거주지     0.654597
서울_거주지     0.656411


In [ ]:
vsm.neighbors('_사용장소', vsm.pmi(b_df))